In [1]:
import FinanceDataReader as fdr
fdr.__version__

'0.9.31'

In [2]:
# 신라젠, 2018년
df = fdr.DataReader('215600', '2018')
df.head(10)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-01-02,95900,104000,93300,102500,6824281,0.096257
2018-01-03,102600,104900,99500,103000,4760150,0.004878
2018-01-04,102600,104000,92200,92200,6538665,-0.104854
2018-01-05,85800,101200,85700,100000,8318372,0.084599
2018-01-08,98000,98400,92500,93800,6352776,-0.062000
2018-01-09,96500,119200,93800,109000,12734902,0.162047
2018-01-10,105000,107300,97800,98000,6588397,-0.100917
2018-01-11,97600,100600,95200,96700,4115386,-0.013265
2018-01-12,99800,103300,97100,98100,6806596,0.014478


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer

In [4]:
code = "005930"

In [5]:
url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"

In [6]:
print(url)

http://finance.naver.com/item/sise_day.nhn?code=005930


In [7]:
headers = urllib.request.urlretrieve('http://finance.naver.com/item/sise_day.nhn?code=005930')

In [8]:
headers

('C:\\Users\\inhwa\\AppData\\Local\\Temp\\tmpbbzf5dwo',
 <http.client.HTTPMessage at 0x29a9a51b760>)

In [116]:
headers = { 
    'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'), 
}
pages_to_fetch = 10

from urllib.request import urlopen
def run():
    url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
    req = urllib.request.Request(url, headers=headers)
    with urlopen(req) as doc:
        if doc is None:
            return None
        html = BeautifulSoup(doc, "lxml")
        pgrr = html.find("td", class_="pgRR")
        if pgrr is None:
            return None
        s = str(pgrr.a["href"]).split('=')
        lastpage = s[-1] 
    df = pd.DataFrame()
    pages = min(int(lastpage), pages_to_fetch)
    print(pages)
    for page in range(1, pages + 1):
        pg_url = '{}&page={}'.format(url, page)
        req = urllib.request.Request(pg_url, headers=headers)
        with urlopen(req) as doc:
            if doc is None:
                return None
            html = BeautifulSoup(doc, "lxml")
            table = html.find_all("table")[0]
            print(table)
            dfs = pd.read_html(table, header=0, encoding='utf-8')[0]
            print(dfs)
            
            
            #df = df.append(pd.read_html(table, header=0)[0])
       # print(df)
       # tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
       # print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
       #     format(tmnow, company, code, page, pages), end="\r")


In [117]:
html_string = """
<table>
  <tr>
    <th>Programming Language</th>
    <th>Creator</th> 
    <th>Year</th>
  </tr>
  <tr>
    <td>C</td>
    <td>Dennis Ritchie</td> 
    <td>1972</td>
  </tr>
  <tr>
    <td>Python</td>
    <td>Guido Van Rossum</td> 
    <td>1989</td>
  </tr>
  <tr>
    <td>Ruby</td>
    <td>Yukihiro Matsumoto</td> 
    <td>1995</td>
  </tr>
</table>
"""

In [118]:
pd.read_html(html_string)[0]

,Programming Language,Creator,Year
0,C,Dennis Ritchie,1972
1,Python,Guido Van Rossum,1989
2,Ruby,Yukihiro Matsumoto,1995


In [119]:
run()

10
<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>
<tr>
<td colspan="7" height="8"></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2021.07.02</span></td>
<td class="num"><span class="tah p11">80,000</span></td>
<td class="num">
<img alt="하락" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_down.gif" style="margin-right:4px;" width="7"/><span class="tah p11 nv01">
				100
				</span>
</td>
<td class="num"><span class="tah p11">80,000</span></td>
<td class="num"><span class="tah p11">80,400</span></td>
<td class="num"><span class="tah p11">79,900</span></td>
<td class="num"><span class="tah p11">8,181,692</span></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2021.07.01</span></td>
<td class="num"><span class="tah p11">80,100</span></td>

TypeError: 'NoneType' object is not callable

In [ ]:

        df = df.append(table)
        tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
        print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
            format(tmnow, company, code, page, pages), end="\r")
    df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
        ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
    df['date'] = df['date'].replace('.', '-')
    df = df.dropna()
    df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
        'diff', 'open', 'high', 'low', 'volume']].astype(int)
    df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
    return df
            

In [21]:
run()

10


ValueError: No tables found